<a href="https://www.kaggle.com/code/experience08/loan-prediction?scriptVersionId=243892937" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eligibility-prediction-for-loan/Loan_Data.csv


# Importing libraries

In [2]:

import pandas as pd 
import numpy as np
import seaborn as sb
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# Load the data

In [ ]:
#Display the top 5 columns
Loan_data=pd.read_csv('/kaggle/input/eligibility-prediction-for-loan/Loan_Data.csv')
Loan_data.head(5)

# Exploratory Data Analysis

### Summary of the data 

In [ ]:
Loan_data.info()

### Statistically description of the dataset

In [ ]:
Loan_data.describe()

### Checking for null values

In [ ]:
Loan_data.isnull().sum()

### Dropping null values

In [ ]:
New_data=Loan_data.dropna()
# Cheching to see if there are still missing values
New_data.isna().sum()

# Label encoding

In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

# List of columns to label encode
cols_to_encode = ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
                  'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
                  'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status']

# Apply label encoding to each column individually
for col in cols_to_encode:
    New_data.loc[:, col] = label_encoder.fit_transform(New_data[col].astype(str))




In [ ]:
#Displaying the columns we have
New_data.columns

In [ ]:
New_data.info()

#  Features and Target Selection

In [ ]:
# Feature selection
Features=['Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',]

In [ ]:
X=New_data[Features]
X.head()

In [ ]:
y = New_data.Loan_Status
y.head()

In [ ]:
print(type(y))

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y.astype(str))

## Handle class imbalance with SMOTE


In [ ]:
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Split data into training and testing data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)


# Train RandomForest model

In [ ]:
Loan_Prediction_Model = RandomForestClassifier(n_estimators=100, random_state=42)

### Fitting the model 

In [ ]:
Loan_Prediction_Model.fit(X_train, y_train)


# Predictions and Evaluation

In [ ]:
y_pred= Loan_Prediction_Model.predict(X_test)


In [ ]:
y_proba = Loan_Prediction_Model.predict_proba(X_test)[:, 1]

### Accuracy score

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


In [ ]:
auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {auc:.2f}')



### Classification report

In [ ]:
print(classification_report(y_test, y_pred))

### Confusion matrix

In [ ]:
confusion_m =confusion_matrix(y_test, y_pred)
confusion_m

In [ ]:
plt.figure(figsize=(5, 5))
sns.heatmap(confusion_m, 
            annot=True, 
            fmt='d', 
            cmap='pink',
            xticklabels=['0(No)', '1(Yes)'],
            yticklabels=['0(No)', '1(Yes)'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Save the model

In [ ]:
import joblib
joblib.dump(Loan_Prediction_Model, "loan_approval_model.pkl")


In [ ]:
#Testing using new data
new_data=(0,0,0,0,7500,0,2500,0,5000,16,1,0)

# changing the input_data to numpy array
new_data_as_numpy_array = np.asarray(new_data)

# reshape the array as we are predicting for one instance
new_data_reshaped = new_data_as_numpy_array.reshape(1,-1)

prediction = loan_approval_model.predict(new_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('You do not qualify for a loan')
else:
  print('you qualify for a loan')